# Scenario 5: Comparison of modelling tools

This notebook is part of the publication "EnzymeML at Work" from Lauterbach et al. 2022 and compares the fitting of a micro-kinetic model (specified in an EnzymeML document) to experimental data (specified in the same EnzymeML document). 

Generation of the EnzymeML document and individual fitting of the data with either PySCeS or COPASI have been dealt with in separate notebooks.

## Prerequisites

For the kinetic modelling to work, you need to have `PyEnzyme`, `Basico` and `PySCeS` installed. This can be achieved with:

```
    !pip install copasi-basico
    !pip install pysces
    !pip install git+git://github.com/EnzymeML/PyEnzyme.git
```

This is **not needed** when running this notebook via **Binder**, as the environment is already set up.

This notebook can be run in Google Colaboratory (Colab), in order to do this, click on the badge: 
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EnzymeML/Lauterbach_2022/blob/main/Scenario5/PySCeS/Model4_PySCeS_vs_COPASI.ipynb)  

Once in Colab, execute the cell below. This will set up the computational environment, install
PyEnzyme and load the dataset from GitHub.

In [ ]:
conda_install_commands = """
pip uninstall -y click

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.11.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.anaconda.com/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install -c conda-forge conda python=3.7 -y
conda update -y -n base -c conda-forge conda
conda update -y -c conda-forge -n base --all
conda install -y -c conda-forge -n base assimulo
pip install pysces
pip install lmfit
pip install python-libsbml
pip install python-libcombine
pip install copasi-basico
pip install git+https://github.com/EnzymeML/PyEnzyme.git
"""

def install_conda():
    import subprocess
    import sys, os
    with open('install_conda.sh', 'w') as f:
        f.write(conda_install_commands)
    subprocess.run(['/bin/bash', './install_conda.sh'])
    sys.path.append('/usr/local/lib/python3.7/site-packages')
    os.environ['PYTHONPATH'] = ''

if 'google.colab' in str(get_ipython()):
    install_conda()
    !wget https://github.com/EnzymeML/Lauterbach_2022/raw/main/Scenario5/PySCeS/Model_4.omex
    !wget https://github.com/EnzymeML/Lauterbach_2022/raw/main/Scenario5/PySCeS/EnzymeML_Lagerman_init_values_.yaml
else:
    print('Not running on Colab.')

For the parameter estimation with PySCeS, the CVODE algorithm is needed; this is provided by **Assimulo**. If you are using the **Anaconda** Python Distribution (and when running this notebook via **Binder**), this can easily be achieved by uncommenting and running the following line of code. Alternatively, refer to the Assimulo documentation: https://jmodelica.org/assimulo/

This is **not needed** for Google Colab, as the install script above already sets up the environment.

In [ ]:
# !conda install -y -c conda-forge assimulo

-----
## Comparison of modelling with PySCeS and COPASI

In separate notebooks, the EnzymeMLDocument that had been adapted to the micro-kinetic model was previously modeled and optimized using PySCeS and COPASI. Since both modeling package interfaces are an integral part of PyEnzyme (linked via an interface called Thin Layer), a simple call to the corresponding Thin Layer object is necessary.

In [ ]:
# Load the EnzymeML Document from file
from pyenzyme import EnzymeMLDocument

enzmldoc = EnzymeMLDocument.fromFile("Model_4.omex")

### Modelling with the PySCeS thin layer

Thin Layers require to follow a given metaclass and thus the syntax of every modeling layer follows the Initialization > ```optimize```-method > ```write```-method procedure. 

In [ ]:
from pyenzyme.thinlayers import ThinLayerPysces

Before optimization, it might be necessary to define initial values. Since manipulating the KineticParameter initial_values attributes inside the script that generates the EnzymeMLDocument can get quite tedious, PyEnzyme offers an external data structure from within initial values can be applied. This way, the EnzymeML document is only modifed at optimization and remains untouched until then.

The initialization file is in the YAML format and contains all reactions and their parameters together with the initial estimates for the parameter valuess. 

In [ ]:
# Initialize the layer
tl_pysces = ThinLayerPysces(
    "Model_4.omex", 
    init_file="EnzymeML_Lagerman_init_values_.yaml",
    model_dir="pySCeS"
)

In [ ]:
# Run optimization
tl_pysces.model.mode_integrator='CVODE'
tl_opt = tl_pysces.optimize(method="least_squares")

# Write to new EnzymeMLDocument and save
pysces_doc = tl_pysces.write()
pysces_doc.toFile(".", name="EnzymeML_Lagerman_M4_PySCeS_Modeled")

### Modelling with the COPASI thin layer

In the same manner the COPASI Thin Layer can be used to model the given data. 

The COPASI optimization is set up to use the same initial values and the same fitting algorithm that was used with PySCeS, to allow an easy comparison.

In [ ]:
from pyenzyme.thinlayers import ThinLayerCopasi

In [ ]:
# Initialize COPASI Thin Layer
tl_copasi = ThinLayerCopasi(
    "Model_4.omex", "COPASI",
    init_file="EnzymeML_Lagerman_init_values_.yaml"
)

tl_copasi.optimize()

In [ ]:
copasi_doc = tl_copasi.write()
copasi_doc.toFile(".", name="EnzymeML_Lagerman_M4_COPASI_Modeled")

### Comparison of results

Both results can now be compared by individually exporting the estimated parameters using the ```exportKineticParameters```-method found in the ```EnzymeMLDocument``` instance that returns a Pandas ```DataFrame``` object. Finally, for the sake of comparison, both result are merged into a single ```DataFrame```.

In [ ]:
params = pysces_doc.exportKineticParameters(exclude_constant=True)
params.rename({"value": "PySCeS"}, axis="columns", inplace=True)
params["COPASI"] = copasi_doc.exportKineticParameters(exclude_constant=True).value

params[["name", "PySCeS", "COPASI", "unit"]]

-------